In [28]:

!pip install tensorflow


In [45]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
from keras.layers import Dense


In [46]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
from keras.layers import Dense

# Load data
train_essays_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train_prompts_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
test_essays_df = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

# Merge data
train_data = pd.merge(train_essays_df, train_prompts_df, on='prompt_id', how='left')

# Extract features and labels
X_train = train_data['text']
y_train = train_data['generated']

# Convert X_train to a list of strings
X_train_list = X_train.astype(str).tolist()

# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_list)

# Split into train and validation sets
X_train_tfidf, X_val_tfidf, y_train, y_val = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

# Define and compile the model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_tfidf.shape[1],)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_tfidf, y_train, epochs=5, batch_size=32, validation_data=(X_val_tfidf, y_val))

# Predict on the validation set
y_val_pred_proba = model.predict(X_val_tfidf).ravel()
roc_auc = roc_auc_score(y_val, y_val_pred_proba)
print(f'Validation ROC AUC: {roc_auc}')

# Predict on the test set
X_test_list = test_essays_df['text'].astype(str).tolist()
X_test_tfidf = tfidf_vectorizer.transform(X_test_list)
y_test_pred_proba = model.predict(X_test_tfidf).ravel()

# Save results to CSV
result_df_nn = pd.DataFrame({'id': test_essays_df['id'], 'generated': y_test_pred_proba})
result_df_nn.to_csv('submission_nn.csv', index=False)









Epoch 1/5


InvalidArgumentError: Graph execution error:

TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/opt/conda/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 266, in __call__
    return func(device, token, args)

  File "/opt/conda/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 144, in __call__
    outputs = self._call(device, args)

  File "/opt/conda/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 151, in _call
    ret = self._func(*args)

  File "/opt/conda/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/data_adapter.py", line 519, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/data_adapter.py", line 519, in <listcomp>
    return [slice_array(